## 2 Functions
1. `parse_for_ingestion`: Parsing for ingestion (more specific and intentional)
2. `parse_user_files`: Parsing for user routing (general, not as important)

In [1]:
def parse_for_ingestion():
    pass

In [2]:
def parse_user_files(fname):
    # handle different file types
    if fname.endswith('.csv'):
        pass
    elif fname.endswith('.xml'):
        pass
    elif fname.endswith('.pptx'):
        pass
    elif fname.endswith('.docx'):
        pass
    elif fname.endswith('.pdf'):
        pass
    elif fname.endswith('.txt'):
        pass